In [26]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
from collections import OrderedDict
import seaborn as sns
import pandas as pd
import gzip
from scipy.signal import filtfilt, butter
import pickle
from pyriemann.estimation import Covariances
from pyriemann.utils.mean import mean_riemann
from pyriemann.utils.distance import distance_riemann
from sklearn.model_selection import KFold
from itertools import combinations,product
from tqdm import tqdm

In [6]:
#Parameters : experimental setup
tmin=2
tmax=5
sfreq = 256
freq_band=0.1
frequencies= [13,17,21]
event_code = [33024,33025,33026,33027]
channels = np.array(['Oz','O1','O2','PO3','POz','PO7','PO8','PO4'])
names=['resting','stim13','stim21','stim17']

In [7]:
#loading data
def organize_data_into_dict(data_path= 'ssvep_dataset/'):
    subj_list = os.listdir(data_path)
    records = {k: [] for k in subj_list}
    for subj in subj_list:
        record_all = os.listdir(data_path+subj+'/')
        n = len(record_all)//4#number of records of a given subject
        for i in range(n):
            records[subj].append(record_all[i*4][:28])
    return records

def load_data(chosen_subject,chosen_index,records,data_path= 'ssvep_dataset/'):
    subj_list = list(records.keys())
    assert chosen_subject in subj_list,"The chosen subject doesn't exist in the dataset."
    assert chosen_index in range(len(records[chosen_subject])),"The chosen record doesn't exist for the subject "+chosen_subject

    fname = chosen_subject+'/'+records[chosen_subject][chosen_index]
    with gzip.open(data_path + fname + '.pz', 'rb') as f:
        o = pickle.load(f, encoding='latin1')
    raw_signal = o['raw_signal'].T
    event_pos = o['event_pos'].reshape((o['event_pos'].shape[0]))
    event_type = o['event_type'].reshape((o['event_type'].shape[0]))
    return raw_signal,event_pos,event_type

In [8]:
#The Butterworth filter : band-pass filter, flat in the passband , the passband is concentrated on 
def filter_bandpass(signal, fmin, fmax, fs, order=4, filttype='forward-backward'):
    nyq = 0.5 * fs
    low = fmin / nyq
    high = fmax / nyq
    b, a = butter(order, [low, high], btype='band')
    #filter tpe : forwaard-backward
    filtered = filtfilt(b, a, signal, axis=-1)  
    return filtered

In [9]:
def extended_trials(raw_signal,event_pos,event_type,sfreq=sfreq,frequencies=frequencies,tmin=tmin,tmax=tmax,
                    freq_band=freq_band,channels = channels):
    ext_signal = np.empty_like(raw_signal[0,:])    
    for f in frequencies:
        ext_signal = np.vstack((ext_signal, filter_bandpass(raw_signal, f-freq_band, f+freq_band, fs=sfreq)))
    ext_signal = ext_signal[1:,:]
    ext_trials = list()
    for e, t in zip(event_type, event_pos):
        if e == 32779: # start of a trial
            start = t + tmin*sfreq
            stop  = t + tmax*sfreq
            ext_trials.append(ext_signal[:, start:stop])
    ext_trials = np.array(ext_trials)
    ext_trials = ext_trials - np.tile(ext_trials.mean(axis=2).reshape(ext_trials.shape[0], 
                                ext_trials.shape[1], 1), (1, 1, ext_trials.shape[2]))
    return ext_trials
    

In [10]:
def make_labels(event_type,event_code=event_code):
    labels = []
    n_events = len(event_code)
    for e in event_type:
        for i in range(n_events):
            if e==event_code[i]:
                labels.append(i)
    return labels

In [11]:
#visualisation of the extended signals of a given trial
def visualisation_of_ext_trails(ext_trials,trial,n_seconds,frequencies=frequencies,sfreq=sfreq,
                               channels = channels):
    n_trials = ext_trials.shape[0]
    assert trial in range(n_trials),"The selected trial is out of range."
    assert n_seconds <=3, "The duration of a trial is 3s. Make sure to select less seconds for visualization"
    n_channels = len(channels)
    time = np.linspace(0, n_seconds, n_seconds * sfreq).reshape((1, n_seconds * sfreq))
    fig, axs = plt.subplots(4,2,figsize=(15,15))
    axs =axs.flatten()
    for i in range(n_channels):
        for j in range(len(frequencies)):
            axs[i].plot(time.T, ext_trials[trial, n_channels*j+i, :].T, label=str(frequencies[j])+' Hz')
        if i%2==0:
            axs[i].set_ylabel("$\\mu$V")
        axs[i].set_title(channels[i])
        axs[i].legend(loc='upper left')
    axs[i-1].set_xlabel('Time (s)')
    axs[i].set_xlabel('Time (s)')

In [12]:
def covariances(ext_trials,estimator='scm'):
    cov_ext_trials = Covariances(estimator='scm').transform(ext_trials)
    return cov_ext_trials

In [13]:
def GeometricCenters(x_train,y_train,nb_classes=len(frequencies)+1):
    cov_centers = np.empty((nb_classes, x_train.shape[1], x_train.shape[1]))
    x_trains=[[] for i in range(nb_classes) ]
    for i in range(nb_classes):
        for j in range(x_train.shape[0]):
            if y_train[j]==i:
                x_trains[i].append(x_train[j,:,:])
    for i in range(nb_classes):
        x_trains[i]=np.asarray(x_trains[i])
    for i in range(nb_classes):
        cov_centers[i, :, :] = mean_riemann(x_trains[i])
    return cov_centers


In [14]:
def accuracy(x,y,cov_centers):
    nb_classes= cov_centers.shape[0]
    classes=list(range(cov_centers.shape[0]))
    accuracies = list()
    for sample, true_label in zip(x, y):
        dist = [distance_riemann(sample, cov_centers[m]) for m in range(nb_classes)]
        if classes[np.array(dist).argmin()] == true_label:
            accuracies.append(1)
        else: accuracies.append(0)
    accuracy_ = 100.*np.array(accuracies).sum()/len(y)
    return accuracy_


In [15]:
def describe_label(y,nb_classes=len(frequencies)+1):
    for i in range(nb_classes):
        count=0
        for z in y:
            if z==i:
                count +=1
        print("Occurence of class ",i," = ",count)

In [ ]:
def random_selection(n,p,q):
    res = []
    i=0
    while i< q:
        element = list(np.random.randint(low = 0, high = n, size = p))
        if not(element in res):
            i +=1
            res.append(element)
    return res

In [29]:
def classify_single_recording_with_balanced_cross_val(chosen_subject,chosen_index,records,k=7,max_folds = 10,n_classes=4):
    #k = number of samples of each class in the training set 
    #for exp, if k=7, the train set contains 4*k=28 samples and thus the test set 4 samples.
    assert 0<k<8, "Incorrect number of samples per class in the train set"
    raw_signal,event_pos,event_type = load_data(chosen_subject,chosen_index,records)
    ext_trials = extended_trials(raw_signal,event_pos,event_type)
    labels = make_labels(event_type)
    cov_ext_trials = covariances(ext_trials)
        
    classes_index = { i :[] for i in range(n_classes)} 
    #assigns for each classs the indices of samples belonging to that class
    
    for j in range(len(labels)) : 
        classes_index[labels[j]].append(j)
        
    #the number of folds is n_classes*C(k,8). So, for k=7, we can build 4*8 = 32 different train and test sets
    
    classes_index_comb = { i :list(combinations(classes_index[i],k)) for i in range(n_classes)} 
    n_folds_per_class = len(classes_index_comb[0]) #=C(k,8)

    possible_train_folds_indx = random_selection(n_folds_per_class,n_classes,max_folds)
        
    trains,tests = [],[]
    for n in range(max_folds):
        train,test = [],[]
        l = possible_train_folds_indx[n]
        for i in range(n_classes):
            train.extend(classes_index_comb[i][l[i]])
        for j in range(len(labels)):
            if not( j in train):
                test.append(j)
        trains.append(train)
        tests.append(test)

    
    train_accuracy, test_accuracy = [], []

    for n in tqdm(range(max_folds)):
        x_train,x_test,y_train,y_test = [],[],[],[]
        
        for i in trains[n]:
            x_train.append(cov_ext_trials[i,:,:])
            y_train.append(labels[i])

        for i in tests[n]:
            x_test.append(cov_ext_trials[i,:,:])
            y_test.append(labels[i])
        
        x_train = np.asarray(x_train)
        x_test  = np.asarray(x_test)
        
        cov_centers = GeometricCenters(x_train,y_train)
        
        train_accuracy.append(accuracy(x_train,y_train,cov_centers))
        test_accuracy.append(accuracy(x_test,y_test,cov_centers))
    
    
    train_accuracy = np.asarray(train_accuracy)
    test_accuracy = np.asarray(test_accuracy)
    
    print("Train Accuracy  = ",round(np.mean(train_accuracy),2),"%  +/- ", round(np.std(train_accuracy),2),"%")
    print("Test  Accuracy  = ",round(np.mean(test_accuracy),2),"%   +/- ", round(np.std(test_accuracy),2),"%")
    return np.mean(train_accuracy),np.mean(test_accuracy)
    

In [31]:
# 24 samples for train
records = organize_data_into_dict()
key_list =[]
for subj in records.keys():
    for session in range(len(records[subj])):
        key_list.append((subj,records[subj][session]))
accuracies_ = { x: {'train accuracy':0,'test accuracy':0} for x in key_list}
for subj in records.keys():
    print("______________Subject : ",subj,"______________")
    for session in range(len(records[subj])):
        print("*****Record : ",records[subj][session],"*****")
        train,test = classify_single_recording_with_balanced_cross_val(subj,session,records,k=6,max_folds = 10)
        accuracies_[(subj,records[subj][session])]['train accuracy'] = train
        accuracies_[(subj,records[subj][session])]['test accuracy'] = test
        print()

______________Subject :  subject01 ______________
*****Record :  record-[2012.07.06-19.02.16] *****


100%|██████████| 10/10 [00:05<00:00,  1.92it/s]


Train Accuracy  =  95.83 %  +/-  2.64 %
Test  Accuracy  =  22.5 %   +/-  12.25 %

*****Record :  record-[2012.07.06-19.06.14] *****


100%|██████████| 10/10 [00:05<00:00,  1.81it/s]


Train Accuracy  =  94.17 %  +/-  2.76 %
Test  Accuracy  =  32.5 %   +/-  13.92 %

______________Subject :  subject02 ______________
*****Record :  record-[2012.07.19-17.36.23] *****


...

Train Accuracy  =  100.0 %  +/-  0.0 %
Test  Accuracy  =  58.75 %   +/-  13.75 %

*****Record :  record-[2014.03.10-20.26.46] *****


100%|██████████| 10/10 [00:05<00:00,  1.84it/s]


Train Accuracy  =  100.0 %  +/-  0.0 %
Test  Accuracy  =  65.0 %   +/-  10.9 %

*****Record :  record-[2014.03.10-20.41.35] *****


100%|██████████| 10/10 [00:05<00:00,  1.85it/s]

Train Accuracy  =  100.0 %  +/-  0.0 %
Test  Accuracy  =  67.5 %   +/-  13.92 %



In [32]:
acc6 = pd.DataFrame(accuracies_)
acc6.T

train accuracy  test accuracy
subject01 record-[2012.07.06-19.02.16]       95.833333          22.50
          record-[2012.07.06-19.06.14]       94.166667          32.50
subject02 record-[2012.07.19-17.36.23]       97.916667          30.00
          record-[2012.07.19-17.41.14]       94.583333          22.50
subject03 record-[2012.07.11-15.25.23]       99.166667          41.25
          record-[2012.07.11-15.33.08]      100.000000          50.00
subject04 record-[2012.07.18-17.52.30]      100.000000          31.25
          record-[2012.07.18-17.56.53]       97.916667          40.00
subject05 record-[2012.07.19-11.24.02]       98.333333          27.50
          record-[2012.07.19-11.28.18]       95.416667          43.75
subject06 record-[2012.07.20-12.20.55]       94.166667          37.50
          record-[2012.07.20-12.26.47]       98.333333          31.25
subject07 record-[2012.07.11-15.33.08]      100.000000          47.50
          record-[2012.07.18-09.15.30]      100.000000          36.25
          record-[2012.07.18-09.21.13]       97.500000          38.75
subject08 record-[2013.04.06-16.22.32]      100.000000          55.00
          record-[2013.04.06-16.29.18]      100.000000          65.00
          record-[2013.04.06-16.35.05]      100.000000          72.50
subject09 record-[2013.04.09-17.32.29]      100.000000          51.25
          record-[2013.04.09-17.39.37]       97.916667          26.25
subject10 record-[2014.02.26-15.10.48]       98.750000          56.25
          record-[2014.02.26-15.32.36]      100.000000          57.50
          record-[2014.02.26-15.40.22]      100.000000          71.25
          record-[2014.02.26-15.50.09]       97.916667          61.25
          record-[2014.02.26-16.18.11]       97.916667          62.50
          record-[2014.02.26-16.25.45]       99.166667          31.25
subject11 record-[2014.02.24-17.56.37]       98.750000          56.25
          record-[2014.02.24-18.02.40]       99.166667          32.50
          record-[2014.02.24-18.15.11]       98.750000          56.25
          record-[2014.02.24-18.23.37]       96.250000          46.25
subject12 record-[2014.03.10-19.17.37]      100.000000          86.25
          record-[2014.03.10-19.47.49]      100.000000          56.25
          record-[2014.03.10-20.11.55]      100.000000          58.75
          record-[2014.03.10-20.26.46]      100.000000          65.00
          record-[2014.03.10-20.41.35]      100.000000          67.50

In [33]:
# 28 samples for train
records = organize_data_into_dict()
key_list =[]
for subj in records.keys():
    for session in range(len(records[subj])):
        key_list.append((subj,records[subj][session]))
accuracies_ = { x: {'train accuracy':0,'test accuracy':0} for x in key_list}
for subj in records.keys():
    print("______________Subject : ",subj,"______________")
    for session in range(len(records[subj])):
        print("*****Record : ",records[subj][session],"*****")
        train,test = classify_single_recording_with_balanced_cross_val(subj,session,records,k=7,max_folds = 10)
        accuracies_[(subj,records[subj][session])]['train accuracy'] = train
        accuracies_[(subj,records[subj][session])]['test accuracy'] = test
        print()

______________Subject :  subject01 ______________
*****Record :  record-[2012.07.06-19.02.16] *****


100%|██████████| 10/10 [00:06<00:00,  1.53it/s]


Train Accuracy  =  92.5 %  +/-  1.92 %
Test  Accuracy  =  27.5 %   +/-  20.77 %

*****Record :  record-[2012.07.06-19.06.14] *****


100%|██████████| 10/10 [00:06<00:00,  1.45it/s]


Train Accuracy  =  95.0 %  +/-  2.86 %
Test  Accuracy  =  30.0 %   +/-  15.0 %

______________Subject :  subject02 ______________
*****Record :  record-[2012.07.19-17.36.23] *****


...

Train Accuracy  =  100.0 %  +/-  0.0 %
Test  Accuracy  =  72.5 %   +/-  17.5 %

*****Record :  record-[2014.03.10-20.26.46] *****


100%|██████████| 10/10 [00:06<00:00,  1.60it/s]


Train Accuracy  =  100.0 %  +/-  0.0 %
Test  Accuracy  =  70.0 %   +/-  21.79 %

*****Record :  record-[2014.03.10-20.41.35] *****


100%|██████████| 10/10 [00:06<00:00,  1.67it/s]

Train Accuracy  =  100.0 %  +/-  0.0 %
Test  Accuracy  =  75.0 %   +/-  15.81 %



In [34]:
acc7 = pd.DataFrame(accuracies_)
acc7.T

train accuracy  test accuracy
subject01 record-[2012.07.06-19.02.16]       92.500000           27.5
          record-[2012.07.06-19.06.14]       95.000000           30.0
subject02 record-[2012.07.19-17.36.23]       97.500000           25.0
          record-[2012.07.19-17.41.14]       93.928571           20.0
subject03 record-[2012.07.11-15.25.23]      100.000000           37.5
          record-[2012.07.11-15.33.08]       99.642857           57.5
subject04 record-[2012.07.18-17.52.30]      100.000000           40.0
          record-[2012.07.18-17.56.53]       99.285714           45.0
subject05 record-[2012.07.19-11.24.02]       97.142857           40.0
          record-[2012.07.19-11.28.18]       91.428571           57.5
subject06 record-[2012.07.20-12.20.55]       94.642857           50.0
          record-[2012.07.20-12.26.47]       98.928571           40.0
subject07 record-[2012.07.11-15.33.08]       99.285714           47.5
          record-[2012.07.18-09.15.30]      100.000000           35.0
          record-[2012.07.18-09.21.13]       99.642857           37.5
subject08 record-[2013.04.06-16.22.32]      100.000000           55.0
          record-[2013.04.06-16.29.18]      100.000000           67.5
          record-[2013.04.06-16.35.05]      100.000000           70.0
subject09 record-[2013.04.09-17.32.29]      100.000000           57.5
          record-[2013.04.09-17.39.37]       96.428571           35.0
subject10 record-[2014.02.26-15.10.48]       98.571429           65.0
          record-[2014.02.26-15.32.36]      100.000000           65.0
          record-[2014.02.26-15.40.22]       99.642857           70.0
          record-[2014.02.26-15.50.09]       97.857143           67.5
          record-[2014.02.26-16.18.11]       97.857143           65.0
          record-[2014.02.26-16.25.45]       98.928571           22.5
subject11 record-[2014.02.24-17.56.37]       94.285714           62.5
          record-[2014.02.24-18.02.40]       98.928571           40.0
          record-[2014.02.24-18.15.11]       99.642857           65.0
          record-[2014.02.24-18.23.37]       96.428571           62.5
subject12 record-[2014.03.10-19.17.37]      100.000000           87.5
          record-[2014.03.10-19.47.49]      100.000000           77.5
          record-[2014.03.10-20.11.55]      100.000000           72.5
          record-[2014.03.10-20.26.46]      100.000000           70.0
          record-[2014.03.10-20.41.35]      100.000000           75.0